In [ ]:
!rm *.csv
!ls

rm: cannot remove '*.csv': No such file or directory
sample_data


In [ ]:
import requests
import pandas as pd
import os
import time
import shutil
import numpy as np
from google.colab import files

def check(array):
    return any(np.isfinite(array))

def delete_file(file_path):
    os.remove(file_path)

Target_Value=[]
daylist=[]
Year, Month = input("輸入:年/月(xxxx/xx)").split("/")
Day_Start, Day_End = input("輸入:開始日/結束日(xx/xx)").split("/")
Target_Value_num = input("輸入要查詢的閘道代碼數量:")
Target_Value_num = int(Target_Value_num)
for i in range(0,Target_Value_num):
  n=i+1
  n=str(n)
  target_value=input("輸入要查詢的閘道代碼"+n+":(xxxxxxxx)")
  Target_Value.append(target_value)
Hour_Start, Hour_End = 0, 23
Target_Field = 'GantryID_O'
OutputDirTag = "110021039"
OutPutDir = "./TDCS_Download_"+OutputDirTag+"/M06A/"+Year+Month

if not (os.path.exists(OutPutDir)):
    os.makedirs(OutPutDir)
    print("Dir Created"+OutPutDir)
else:
    print("Dir Existed"+OutPutDir)

Day_Start, Day_End = int(Day_Start), int(Day_End)
TDCS_M06A = "https://tisvcloud.freeway.gov.tw/history/TDCS/M06A/"
FieldNames = ['VehicleType', 'DetectionTime_O', 'GantryID_O', 'DetectionTime_D', 'GantryID_D', 'TripLength', 'TripEnd', 'TripInformation']
FieldNames_Selected = ['VehicleType', 'DetectionTime_O', 'GantryID_O', 'DetectionTime_D', 'GantryID_D', 'TripLength', 'TripEnd']
total_d = pd.DataFrame(columns=['theday', 'GantryID_O', 'sum_5', 'sum_31', 'sum_32', 'sum_41', 'sum_42'])
sum_d = pd.DataFrame(columns=['theday', 'GantryID_O', '5', '31', '32', '41', '42'])
d = pd.DataFrame(columns=['TheDayTime', 'GantryID_O', '5', '31', '32', '41', '42'])
df = pd.DataFrame(FieldNames_Selected)

for Day in range(Day_Start, Day_End+1):
    if Day < 10:
        DayStr = "0"+str(Day)
        print("0"+str(Day))
    else:
        DayStr = str(Day)
        print(str(Day))

    for Hour in range(Hour_Start, Hour_End+1):
        if Hour < 10:
            HourStr = "0"+str(Hour)
            print("0"+str(Hour))
        else:
            HourStr = str(Hour)
            print(str(Hour))

        OneDate = Year+Month+DayStr
        thedaytime = Year+"/"+Month+"/"+DayStr+"-"+HourStr

        for i in range(1, Target_Value_num+1):
            theTarget_Value = str(i)
            d = pd.concat([d, pd.DataFrame([{'TheDayTime': thedaytime, 'GantryID_O': theTarget_Value}])], ignore_index=True)

        OneDate_OneHour_FileName = "TDCS_M06A_"+OneDate+"_"+HourStr+"0000.csv"
        url = TDCS_M06A+OneDate+'/'+HourStr+'/'+OneDate_OneHour_FileName
        all_df = pd.read_csv(url, encoding='utf-8', header=None, names=FieldNames)
        all_df['DetectionTime_O'] = all_df['DetectionTime_O'].astype('datetime64[ns]')
        all_df['DetectionTime_O'] = all_df['DetectionTime_O'].dt.strftime("%Y/%m/%d-%H")
        OutputFileLocation = OutPutDir+"/"+OneDate_OneHour_FileName
        delDir = "/content/TDCS_Download_"+OutputDirTag+"/M06A/"+Year+Month+"/"+OneDate_OneHour_FileName
        all_df.to_csv(OutputFileLocation)
        print("Download=>"+OutputFileLocation)

        for i in range(1, Target_Value_num+1):
            theTarget_Value = str(i)
            new_df = all_df[(all_df[Target_Field] == theTarget_Value)]
            df = pd.concat([df, new_df], ignore_index=True)
            dff = df.groupby(['DetectionTime_O', 'VehicleType', 'GantryID_O']).size().reset_index(name='counts')
            the5 = (dff['DetectionTime_O'] == thedaytime) & (dff['VehicleType'] == 5) & (dff['GantryID_O'] == theTarget_Value)
            the31 = (dff['DetectionTime_O'] == thedaytime) & (dff['VehicleType'] == 31) & (dff['GantryID_O'] == theTarget_Value)
            the32 = (dff['DetectionTime_O'] == thedaytime) & (dff['VehicleType'] == 32) & (dff['GantryID_O'] == theTarget_Value)
            the41 = (dff['DetectionTime_O'] == thedaytime) & (dff['VehicleType'] == 41) & (dff['GantryID_O'] == theTarget_Value)
            the42 = (dff['DetectionTime_O'] == thedaytime) & (dff['VehicleType'] == 42) & (dff['GantryID_O'] == theTarget_Value)
            checkright = (d['TheDayTime'] == thedaytime) & (d['GantryID_O'] == theTarget_Value)
            VehicleType5 = []
            VehicleType31 = []
            VehicleType32 = []
            VehicleType41 = []
            VehicleType42 = []
            result5 = dff.loc[the5, 'counts'].values
            result31 = dff.loc[the31, 'counts'].values
            result32 = dff.loc[the32, 'counts'].values
            result41 = dff.loc[the41, 'counts'].values
            result42 = dff.loc[the42, 'counts'].values
            resultthe5 = check(result5)
            if resultthe5:
              VehicleType5 = result5[0]
              d.loc[checkright, '5'] = int(VehicleType5)
            else:
              d.loc[checkright, '5'] = int(0)
            resultthe31 = check(result31)
            if resultthe31:
              VehicleType31 = result31[0]
              d.loc[checkright, '31'] = int(VehicleType31)
            else:
              d.loc[checkright, '31'] = int(0)
            resultthe32 = check(result32)
            if resultthe32:
              VehicleType32 = result32[0]
              d.loc[checkright, '32'] = int(VehicleType32)
            else:
              d.loc[checkright, '32'] = int(0)
            resultthe41 = check(result41)
            if resultthe41:
              VehicleType41 = result41[0]
              d.loc[checkright, '41'] = int(VehicleType41)
            else:
              d.loc[checkright, '41'] = int(0)
            resultthe42 = check(result42)
            if resultthe42:
              VehicleType42 = result42[0]
              d.loc[checkright, '42'] = int(VehicleType42)
            else:
              d.loc[checkright, '42'] = int(0)
        time.sleep(10)
        delete_file(delDir)
output_csv = 'output_csv'
if not os.path.exists(output_csv):
  os.makedirs(output_csv)
else:
  shutil.rmtree(output_csv)
  os.makedirs(output_csv)# 將日期和小時分割出來
# 將日期和小時分割出來
d[['theday', 'hour']] = d['TheDayTime'].str.split('-', 1, expand=True)

# 初始化一些變數
total_d = pd.DataFrame(columns=['theday', 'GantryID_O', 'sum_5', 'sum_31', 'sum_32', 'sum_41', 'sum_42'])

# 處理每一天的資料
for i in range(0, len(daylist)):
    dayname = daylist[i]
    dayname = str(dayname)
    ssum_d = d[(d['theday'] == daylist[i])]
    sum_d = pd.concat([sum_d, ssum_d], ignore_index=True)
    for i in range(0, Target_Value_num):
        theTarget_Value = Target_Value[i]
        theTarget_Value = str(theTarget_Value)
        total_d = pd.concat([total_d, pd.DataFrame([{'theday': dayname, 'GantryID_O': theTarget_Value}])], ignore_index=True)
        checkright = (total_d['theday'] == dayname) & (total_d['GantryID_O'] == theTarget_Value)
        sum5 = (sum_d['theday'] == dayname) & (sum_d['GantryID_O'] == theTarget_Value)
        sum_5 = sum_d[sum5]['5'].sum()
        total_d.loc[checkright, 'sum_5'] = int(sum_5)
        sum31 = (sum_d['theday'] == dayname) & (sum_d['GantryID_O'] == theTarget_Value)
        sum_31 = sum_d[sum31]['31'].sum()
        total_d.loc[checkright, 'sum_31'] = int(sum_31)
        sum32 = (sum_d['theday'] == dayname) & (sum_d['GantryID_O'] == theTarget_Value)
        sum_32 = sum_d[sum32]['32'].sum()
        total_d.loc[checkright, 'sum_32'] = int(sum_32)
        sum41 = (sum_d['theday'] == dayname) & (sum_d['GantryID_O'] == theTarget_Value)
        sum_41 = sum_d[sum41]['41'].sum()
        total_d.loc[checkright, 'sum_41'] = int(sum_41)
        sum42 = (sum_d['theday'] == dayname) & (sum_d['GantryID_O'] == theTarget_Value)
        sum_42 = sum_d[sum42]['42'].sum()
        total_d.loc[checkright, 'sum_42'] = int(sum_42)

# 建立每日總車流量的 CSV 檔案
total_dataname = Year + "_" + Month + "_" + str(Day_Start) + "到" + str(Day_End) + "日 每日總車流量.csv"
total_d.to_csv(total_dataname, index=False)
shutil.move(total_dataname, output_csv)

# 處理每個閘道代碼的資料
for i in range(0, Target_Value_num):
    theTarget_Value = Target_Value[i]
    theTarget_Value = str(theTarget_Value)
    nd = d[(d[Target_Field] == theTarget_Value)]
    nd[['theday', 'hour']] = nd['TheDayTime'].str.split('-', 1, expand=True)
    ndcolumns = ['TheDayTime', 'theday', 'hour', 'GantryID_O', '5', '31', '32', '41', '42']
    nd = nd.reindex(columns=ndcolumns)
    nd = nd.drop(['GantryID_O', 'TheDayTime'], axis=1)
    dataname = Year + "_" + Month + "_" + str(Day_Start) + "到" + str(Day_End) + "日" + theTarget_Value + "車流量統計.csv"
    nd.to_csv(dataname, index=False)
    shutil.move(dataname, output_csv)

# 將 CSV 檔案打包成壓縮檔
shutil.make_archive('閘道車流量統計', 'zip', output_csv)
files.download('閘道車流量統計.zip')

KeyboardInterrupt: Interrupted by user